In [ ]:
import datetime
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from geopy.distance import geodesic
from geopy.distance import distance

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")

# Mount drive so that we can access google drive files
from google.colab import drive
drive.mount('/drive')

#cab_data = pd.read_parquet('/drive/My Drive/Colab Notebooks/MMA 830 - Pricing Analytics/yellow_tripdata_2022-06.parquet')
#july = pd.read_parquet('/drive/My Drive/Colab Notebooks/MMA 830 - Pricing Analytics/yellow_tripdata_2022-07.parquet')
#cab_data = pd.read_parquet('/drive/My Drive/Colab Notebooks/MMA 830 - Pricing Analytics/yellow_tripdata_2022-08.parquet')
cab_data = pd.read_parquet('/drive/My Drive/Colab Notebooks/MMA 830 - Pricing Analytics/yellow_tripdata_2021-12.parquet')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
#cab_data = pd.concat([june,july,august],axis=0)
cab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214369 entries, 0 to 3214368
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [ ]:
print(cab_data.isnull().sum())
cab_data = cab_data.dropna()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          102371
trip_distance                 0
RatecodeID               102371
store_and_fwd_flag       102371
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     102371
airport_fee              102371
dtype: int64


In [ ]:
cab_data

In [ ]:
# Identify if date falls on weekend or weekday
cab_data['date'] = cab_data['tpep_pickup_datetime'].astype(str).str[:-9].str.strip()
cab_data['year'] = cab_data['date'].str[:4]
cab_data['month'] = cab_data['date'].str[5:7]
cab_data['day'] = cab_data['date'].str[8:10]
cab_data['day_code'] = pd.to_datetime(cab_data['date'], format='%Y-%m-%d').dt.dayofweek
cab_data['day_type'] = np.where(cab_data['day_code'].isin([5,6]),'weekend', "weekday") # <-- this uses those numbers to group each day

# Identify if datetime is during the day or evening
cab_data['time'] = cab_data['tpep_pickup_datetime'].astype(str).str[-9:].str.strip()
cab_data['hour'] = cab_data['time'].str[:2].astype(int)
cab_data['time_type'] = np.where((cab_data['hour'] >= 8) & (cab_data['hour'] < 17),'Day', "Evening")

# Get length of cab ride
cab_data['length_minutes'] = (cab_data['tpep_dropoff_datetime'] - cab_data['tpep_pickup_datetime']).dt.total_seconds()/60

# Create combined segment column to get unique segments
cab_data['segment'] = cab_data['day_type']+"_"+cab_data['time_type']

In [ ]:
cab_data['cost_per_min'] = cab_data['fare_amount']/cab_data['length_minutes']
cab_data['cost_per_dist'] = cab_data['trip_distance']/cab_data['length_minutes']

In [ ]:
cab_data.isna().sum()
cab_data = cab_data.dropna()

In [ ]:
cab_data = cab_data[(cab_data['trip_distance'] > 0) & (cab_data['trip_distance'] < 10)]
cab_data = cab_data[cab_data['length_minutes'] > 0]

In [ ]:
# Get average price per mile or km for each day and segment
avg_prices = cab_data.groupby(['segment','date'])['trip_distance'].mean().reset_index()

# Get number of trips for each day and segment
demands = cab_data.groupby(['segment','date']).size().reset_index()

data_to_model = avg_prices.merge(demands, how='left', left_on=['segment','date'], right_on=['segment','date'])

data_to_model = data_to_model.rename(columns={"segment":"segment",
                                              "date":"date",
                                              "trip_distance": "avg_distance",
                                              0:"demand"})

#data_to_model['revenue_per_min'] = data_to_model['mean_cost_per_min'] * data_to_model['demand']
data_to_model = data_to_model.sort_values(by=['segment'],ascending=[True])

In [ ]:
data_to_model

,segment,date,avg_distance,demand
0,weekday_Day,2008-12-31,0.840000,1
27,weekday_Day,2022-01-17,0.686667,3
26,weekday_Day,2022-01-03,1.446667,3
25,weekday_Day,2021-12-31,2.335595,30703
24,weekday_Day,2021-12-30,2.301152,37258
...,...,...,...,...
71,weekend_Evening,2021-12-18,2.397879,49722
72,weekend_Evening,2021-12-19,2.498899,36492
73,weekend_Evening,2021-12-25,2.562200,17225
74,weekend_Evening,2021-12-26,2.628917,20671


In [ ]:
# get unique segments
segments = data_to_model['segment'].unique()

# for each segment...
for segment in segments:

  # filter to get only this segment's data
  segment_data = data_to_model[data_to_model['segment'] == segment]

  # Separate prices and demand
  x_prices = segment_data[['mean_cost_per_min']]
  y_demands = segment_data[['demand']]

  # fit linear model
  segment_model = LinearRegression().fit(x_prices, y_demands)
  model_coef = segment_model.coef_[0][0]
  model_intercept = segment_model.intercept_[0]
  model_r2 = segment_model.score(x_prices, y_demands)

  print('------------------------------------')
  print('processing segment...',segment)
  print('------------------')
  print("Coefficient:",model_coef)
  print("Intercept:",model_intercept)
  print("R-squared:",model_r2)

  # Use coefficient and intercept to predict demand and calculate predicted revenue
  segment_data['demand_prediction'] = model_intercept + model_coef * segment_data['mean_cost_per_min']
  segment_data['revenue_prediction'] = segment_data["mean_cost_per_min"] * segment_data["demand_prediction"]

  actual_revenue = round(segment_data['revenue_per_min'].sum(),2)
  predict_revenue = round(segment_data['revenue_prediction'].sum(),2)
  change_in_revenue = predict_revenue - actual_revenue
  
  print('\n')
  print("Actual Revenue per Mile: $"+str(actual_revenue))
  print("Predicted Revenue per Mile: $"+str(predict_revenue))
  print("Change in Revenue: $"+str(change_in_revenue))
  print('\n')

  def custom_solver():
    output = pd.DataFrame()
    for price in np.arange(0.00,5.00,0.01):
      segment_data['solver_demand_pred'] = model_intercept + model_coef * price
      segment_data['solver_revenue_pred'] = price * segment_data["solver_demand_pred"]
      revenue = round(segment_data['solver_revenue_pred'].sum(),2)

      results = pd.DataFrame({"Price":[price],"Revenue":[revenue]})

      output = pd.concat([output, results], axis=0)

    output = output.sort_values(by=['Revenue'],ascending=False)
    optimal_price = output['Price'].iat[0]
    revenue_wOptimalPrice = output['Revenue'].iat[0]

    return optimal_price, revenue_wOptimalPrice

  optimal_price, revenue_using_optimalPrice = custom_solver()
  optimal_price = round(optimal_price,2)
  revenue_using_optimalPrice = round(revenue_using_optimalPrice,2)
  revenue_change_using_optimal = revenue_using_optimalPrice - actual_revenue

  print("Optimal price for this segment: $"+str(optimal_price))
  print("Revenue using optimal price for this segment: $"+str(revenue_using_optimalPrice))
  print("Change in revenue (using optimal vs not using optimal): $"+str(revenue_change_using_optimal))
  print("\n")